# Geospatial Map of Germany

This Jupyter Notebook contains code to demonstrate a potential visualization (a geospatial choropleth map) of standardized inpatient treatment costs of a selected ICD-10-diagnosis across german federal lands ("Bundesländer").

A similar graph at other administrative levels (e.g. german distrcits) can be easily made by changing a few lines of the code. I identified the following necessary tasks:


In [ ]:
x <- c(1, 2, 3, 10)
print(sum(x))
